In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import pandas as pd

### Simple models
In this file, we tried to train our model very simply, without advanced techniques to encode the tweets.

In [2]:
from cooc import *
#rom glove_template import *
from pickle_vocab import *

In [3]:
# creating training data
with open('twitter-datasets/train_pos.txt') as f:
    pos = f.readlines()
    
with open('twitter-datasets/train_neg.txt') as f:
    neg = f.readlines()
    
train=pos+neg
train_target=[':)']*len(pos)+[':(']*len(neg)

In [4]:
full_train={'data':train,
           'target':train_target}
full_train=pd.DataFrame(full_train)
full_train.sample(5)

,data,target
172568,<user> i had to resign before they fired me\n,:(
133949,i wish i was in daytona with my friends #sadtw...,:(
73852,now . just gotta sneak into the school\n,:)
92739,""" <user> lmfaooo , now drinking with my bitch ...",:)
19009,<user> ready ... set ... here we go ! ! ! <use...,:)


### Bayes Classifier

In [6]:
from sklearn.model_selection import train_test_split

train , test=train_test_split(full_train)

In [7]:
# Extracting features from text files
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.data)
X_train_counts.shape

(150000, 85161)

In [8]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(150000, 85161)

In [9]:
# Training Naive Bayes (NB) classifier on training data.
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, train.target)

In [10]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

text_clf = text_clf.fit(train.data, train.target)

In [13]:
predicted = text_clf.predict(test.data)
print('Accuracy of Naive Bayes CLassifier: ', np.mean(predicted == test.target))

Accuracy of Naive Bayes CLassifier:  0.76564


### SVM

In [14]:
from sklearn.linear_model import SGDClassifier

text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-5, max_iter=100, random_state=42))])

text_clf_svm = text_clf_svm.fit(train.data, train.target)
predicted_svm = text_clf_svm.predict(test.data)
print('Accuracy of Support Vector Machine model: ',np.mean(predicted_svm == test.target))

Accuracy of Support Vector Machine model:  0.81362


For both models, we got quite good accuracy, especially for SVM. Theses models are interesting as they are very simple and efficient.